# Demo: Serenity Data Download API

Basic demonstration of how to get URL and token to list and download Serenity's files.

## Setup

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import pandas as pd
import seaborn as sns

# set Seaborn style
sns.set(style="whitegrid")

# create an alias to the api
api = connect_widget.get_api()

## CTM Projection Rate Dataset

In [ ]:
# Generate URL and Token to access CTM Projection Rate dataset

from serenity_types.storage.core import DatasetContent

dataset_url_and_token = api.storage_api.get_ctm_download_url_and_token(DatasetContent.CTM_PROJ_RATE)
dataset_url_and_token

#### List all of the available files

In [ ]:
import requests
from xml.etree import ElementTree

# Access the generated URL + token to get list of available files
list_files = requests.get(dataset_url_and_token.full_list_files_url)
list_files_tree = ElementTree.fromstring(list_files.content)

all_filenames = [file.text for file in list_files_tree.findall(".//Name")]
all_filenames

#### Download files and reconstruct as DataFrame

In [ ]:
from io import BytesIO

ctm_proj_rates = []

# Download only the first 5 files
for filename in all_filenames[:5]:
    download_file_url = f"{dataset_url_and_token.root_url}/{filename}?{dataset_url_and_token.sas_token}"
    downloaded_file = requests.get(download_file_url)
    df = pd.read_parquet(BytesIO(downloaded_file.content))
    ctm_proj_rates.append(df)

ctm_proj_rates_df = pd.concat(ctm_proj_rates)
ctm_proj_rates_df

## CTM Basis Dataset

In [ ]:
# Generate URL and Token to access CTM Basis dataset

from serenity_types.storage.core import DatasetContent

ctm_basis_url_and_token = api.storage_api.get_ctm_download_url_and_token(DatasetContent.CTM_BASIS)
ctm_basis_url_and_token

#### List all of the available files

In [ ]:
import requests
from xml.etree import ElementTree

# Access the generated URL + token to get list of available files
list_files = requests.get(ctm_basis_url_and_token.full_list_files_url)
list_files_tree = ElementTree.fromstring(list_files.content)

all_filenames = [file.text for file in list_files_tree.findall(".//Name")]
all_filenames

#### Download files and reconstruct as DataFrame

In [ ]:
from io import BytesIO

ctm_basis = []

# Download only the first 5 files
for filename in all_filenames[:5]:
    download_file_url = f"{ctm_basis_url_and_token.root_url}/{filename}?{ctm_basis_url_and_token.sas_token}"
    downloaded_file = requests.get(download_file_url)
    df = pd.read_parquet(BytesIO(downloaded_file.content))
    ctm_basis.append(df)

ctm_basis_df = pd.concat(ctm_basis)
ctm_basis_df